In [ ]:
import pandas as pd
import os
import re
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
!apt update
!pip install transformers datasets phonemizer -y
!apt install espeak -y
!pip install pydub -y
!pip install transformers --upgrade -y
!pip install torchaudio -y
!pip install tqdm --upgrade -y
!pip install torchaudio --upgrade -y

In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

In [ ]:
from transformers import __version__ as transformers_ver
from tqdm import __version__ as tqdm_ver
from torch import __version__ as torch_ver
from torchaudio import __version__ as torchaudio_ver
from pandas import __version__ as pd_ver
print(f"transformers_ver:\t{transformers_ver}")
print(f"tqdm_ver:\t{tqdm_ver}")
print(f"torch_ver:\t{torch_ver}")
print(f"torchaudio_ver:\t{torchaudio_ver}")
print(f"pandas_ver:\t{pd_ver}")

In [ ]:
!pip install phonemizer

In [ ]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

# model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-xlsr-53-espeak-cv-ft")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-xlsr-53-espeak-cv-ft")

In [ ]:
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-xlsr-53-espeak-cv-ft")

In [ ]:
df = pd.read_csv('/kaggle/input/tat-asr/tat/asr_tat.csv')
df

In [ ]:
df['fpath'] = df['file_name'].map(lambda x: '/kaggle/input/tat-asr/tat/' + x)
df['fpath']

In [ ]:
import torch
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
import torch
import torchaudio
from tqdm.auto import tqdm
tqdm.pandas()
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

def recognizer(fpath):
    try:
        waveform, sample_rate = torchaudio.load(fpath)
        waveform = waveform.to(device)
        logits = model(waveform).logits
        pred_ids = torch.argmax(logits, dim=-1)
        pred_str = processor.batch_decode(pred_ids)[0]
        return pred_str
    except:
        return 0

In [ ]:
df['recognized'] = df['fpath'].progress_apply(recognizer)

In [ ]:
df['recognized'][0]

In [ ]:
!pip install abydos

import abydos

from abydos import distance

phonetic = distance.PhoneticEditDistance()

In [ ]:
phonetic = distance.PhoneticEditDistance()
def phonetic_metric(row):
    try:
        result = phonetic.dist(row['transcription'], row['recognized'])
        return result
    except Exception as e:
        print(e)

In [ ]:
df['phonetic_ev'] = df.progress_apply(phonetic_metric, axis=1)

In [ ]:
df[['transcription', 'recognized', 'phonetic_ev']].sample(10)

In [ ]:
df.phonetic_ev.plot.hist(bins=20)

In [ ]:
df.phonetic_ev.mean()

In [ ]:
def recognizer_adv(fpath):
    try:
        waveform, sample_rate = torchaudio.load(fpath)
        waveform = waveform.to(device)
        logits = model(waveform).logits
        pred_ids = torch.argmax(logits, dim=-1)
        pred_str = processor.batch_decode(pred_ids, uselexicon=true)[0]
        return pred_str
    except:
        return 0

In [ ]:
def phonetic_metric_adv(row):
    try:
        result = phonetic.dist(row['transcription'], row['recognized_adv'])
        return result
    except Exception as e:
        print(e)

In [ ]:
df['phonetic_ev_adv'] = df.progress_apply(phonetic_metric_adv, axis=1)

In [ ]:
df[['transcription', 'recognized_adv', 'phonetic_ev_adv']].sample(10)